In [5]:
import xml.etree.ElementTree as ET
import re
import os.path
from io import BytesIO

#tree = ET.parse(r'C:\Users\Chris\Documents\projects\train-game\drawing.svg')
infile = 'drawing.svg'
svg_outfile = os.path.join('.', 'public', 'ticket-nobg.svg')
react_outfile = os.path.join('.', 'public', 'ticket-nobg.react.svg')

ns = {'svg': 'http://www.w3.org/2000/svg',
      'ttr': 'https://train-game.github.io',
          'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('inkscape', 'http://www.inkscape.org/namespaces/inkscape')
ET.register_namespace('sodipodi', 'http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd')

def fix_doc(node):
    # Remove namespaced attributes from node
    for attrib_name in list(node.attrib.keys()):
        if 'inkscape' in attrib_name or 'sodipodi' in attrib_name or 'XML' in attrib_name:
            del node.attrib[attrib_name]
    for child in node.findall('*'):
        # Strip nodes
        if 'inkscape' in child.tag or 'sodipodi' in child.tag or child.get('id') == 'BG':
            node.remove(child)
            continue

        # Rename inkscape:label to id
        label = child.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label')
        if label:
            child.set('id', label)
            del child.attrib['{http://www.inkscape.org/namespaces/inkscape}label']

    for child in node.findall('*'):
        fix_doc(child)

def add_additional_data_to_routes(node):
    for route in node.find("*/[@id='Routes']").findall('*'): # All first-level routes (single routes + double routes)
        for route in 
    

def make_child_route_ids(node):
    for route in node.find("*/[@id='Routes']").findall('*'):
        print(route)
        for ix, child in enumerate(route.findall('{http://www.w3.org/2000/svg}g')):
            child.set('id', f"{route.attrib['id']}:{ix}")
            
def make_reacty(node):
    # Make styles react-y (part 1)
    style = node.attrib.get('style')
    if style:
        rules = style.split(';')
        new_rules = []
        for rule in rules:
            key, value = rule.split(':')
            new_key = re.sub(r'(-\w)', lambda result: result.group(1).replace('-', '').upper(), key)
            new_rules.append(f'{new_key}:"{value}"')
        
        node.attrib['style'] = '{{' + ','.join(new_rules) + '}}'
    
    for child in node.findall('*'):
        make_reacty(child)

def to_react_string(svg_node):
    svg_data = ET.tostring(root).decode()
    svg_data = re.sub(r'style="(.*?)"', lambda result: f"style={result.group(1)}", svg_data)
    svg_data = svg_data.replace(r'&quot;', '"')
    return svg_data  # svg_data[:1000]


def add_on_click_to_routes(root):
    for route in root.findall('#Routes>g'):
        pass


In [4]:
tree = ET.parse(infile)

root = tree.getroot()

fix_doc(root)
make_child_route_ids(root)
# write fixed file
open(svg_outfile,'w').write(ET.tostring(root).decode())

# write reacty file
make_reacty(root)
open(react_outfile, 'w').write(to_react_string(root))

<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091F9D00>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091F9F80>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FA3E0>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FA6B0>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FAD90>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FAF20>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FB100>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FB380>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FB650>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FB830>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FBAB0>
<Element '{http://www.w3.org/2000/svg}g' at 0x00000153091FBC90>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000015309200040>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000015309200270>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000015309200590>
<Element '{http://www.w3.org/2000/svg}g'

95098

In [20]:
tree = ET.parse(infile)
root = tree.getroot()
fix_doc(root)
# make_child_route_ids(root)

ROUTE_ID_FORMAT_STRING = '{to_and_from}:{lane_index}:{color}:{length}'
for route in root.find("*/[@id='Routes']").findall('*'): # All first-level routes (single routes + double routes)
    num_trains = len(route.findall('{http://www.w3.org/2000/svg}rect'))
    color = route.attrib.get('class', None)
    to_and_from = route.attrib['id']

    if num_trains > 0:
        if not color:
            color = "grey"
        new_id = ROUTE_ID_FORMAT_STRING.format(to_and_from=to_and_from, lane_index=0, color=color, length=num_trains)
        print(new_id)
        route.set('id', new_id)
    else:
        for ix, child in enumerate(route.findall('{http://www.w3.org/2000/svg}g')):
            num_trains = len(child.findall('{http://www.w3.org/2000/svg}rect'))
            if not color:
                color = child.attrib.get('class', 'grey')  # cascade
            new_id = ROUTE_ID_FORMAT_STRING.format(to_and_from=to_and_from, lane_index=ix, color=color, length=num_trains)
            print(new_id)
            child.set('id', new_id)


vancouver-calgary:0:grey:3
seattle-helena:0:yellow:6
helena-duluth:0:orange:6
san_francisco-salt_lake_city:0:grey:5
san_francisco-salt_lake_city:1:grey:5
calgary-helena:0:grey:4
helena-denver:0:grey:4
oklahoma_city-kansas_city:0:grey:2
oklahoma_city-kansas_city:1:grey:2
kansas_city-saint_louis:0:grey:2
kansas_city-saint_louis:1:grey:2
omaha-kansas_city:0:grey:1
omaha-kansas_city:1:grey:1
omaha-duluth:0:grey:2
omaha-duluth:1:grey:2
helena-omaha:0:grey:5
salt_lake_city-denver:0:grey:3
salt_lake_city-denver:1:grey:3
montreal-boston:0:grey:2
montreal-boston:1:grey:2
new_york-boston:0:grey:2
new_york-boston:1:grey:2
montreal-new_york:0:grey:3
raleigh-washington:0:grey:2
raleigh-washington:1:grey:2
atlanta-raleigh:0:grey:2
atlanta-raleigh:1:grey:2
sault_st_marie-toronto:0:grey:2
duluth-sault_st_marie:0:grey:3
winnipeg-duluth:0:grey:4
winnipeg-sault_st_marie:0:grey:6
santa_fe-denver:0:grey:2
houston-new_orleans:0:grey:2
saint_louis-nashville:0:grey:2
little_rock-saint_louis:0:grey:2
oklahoma_